In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

import random
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample


## Reading and preprocessing data

In [5]:
x_train = pd.read_csv('train.csv', index_col='ID')
x_test = pd.read_csv('test.csv', index_col='ID')
y_train = x_train['y']
x_train.drop('y', axis=1, inplace=True)

le = LabelEncoder()

In [3]:
def str_to_int(df, col_name):
    if df[col_name].dtype != 'int64':
        le.fit(df[col_name].values)
        df[col_name] = le.transform(df[col_name])

for col_name in x_train.columns:
    str_to_int(x_train, col_name)
    str_to_int(x_test, col_name)

### Feature selection and generation

In [4]:
selector = SelectKBest(k=75, score_func=f_regression).fit(x_train, y_train)
x_train_new = selector.transform(x_train)
x_test_new = selector.transform(x_test)

/Users/krybkin/anaconda/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:279: RuntimeWarning: invalid value encountered in true_divide
  corr /= row_norms(X.T)
/Users/krybkin/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/krybkin/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/krybkin/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [5]:
from sklearn.decomposition import PCA, FastICA
n_comp = 10

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(x_train)
ica2_results_test = ica.transform(x_test)

x_train_new = pd.DataFrame(x_train_new)
x_test_new = pd.DataFrame(x_test_new)

# Append decomposition components to datasets
for i in range(1, n_comp+1):
    x_train_new['pca_' + str(i)] = pca2_results_train[:,i-1]
    x_test_new['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    x_train_new['ica_' + str(i)] = ica2_results_train[:,i-1]
    x_test_new['ica_' + str(i)] = ica2_results_test[:, i-1]
    
y_mean = np.mean(y_train)


/Users/krybkin/anaconda/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [7]:
# Append decomposition components to datasets
for i in range(1, n_comp+1):
    x_train_new['pca_' + str(i)] = pca2_results_train[:,i-1]
    x_test_new['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    x_train_new['ica_' + str(i)] = ica2_results_train[:,i-1]
    x_test_new['ica_' + str(i)] = ica2_results_test[:, i-1]
    
y_mean = np.mean(y_train)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(x_train_new, y_train, random_state=42, train_size=.95)

### Training model

In [8]:
import xgboost as xgb

xgb_params = {
    'n_trees': 374, 
    'eta': 0.025,
    'gamma': 0.55,
    'max_depth': 2,
    'subsample': 0.8,
    'min_child_weight': 2,
    'colsample_bytree': 0.8,
    'objective': 'reg:linear',
    'nthread': 6,
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}
# xgb_params = {'colsample_bytree': 0.65, 'eta': 0.30000000000000004, 'eval_metric': 'rmse', 'gamma': 0.9500000000000001, 'max_depth': 8, 'min_child_weight': 1.0, 'n_estimators': 110.0, 'nthread': 6, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.9}

dtrain = xgb.DMatrix(x_train_new, y_train)
dtest = xgb.DMatrix(x_test_new)

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=1500,
                   early_stopping_rounds=150,
                   verbose_eval=50, 
                   show_stdv=False
                  )

num_boost_rounds = len(cv_result)
print(num_boost_rounds)

# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)


[0]	train-rmse:12.5191	test-rmse:12.5179
[50]	train-rmse:8.99811	test-rmse:9.00387
[100]	train-rmse:8.43949	test-rmse:8.47981
[150]	train-rmse:8.2447	test-rmse:8.39429
[200]	train-rmse:8.09566	test-rmse:8.372
[250]	train-rmse:7.9588	test-rmse:8.36543
[300]	train-rmse:7.85318	test-rmse:8.36954
[350]	train-rmse:7.75878	test-rmse:8.37366
251


## Tuning hyperparameters

In [9]:
def score(params):
    print("Training with params : ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    # watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    model = xgb.train(params, dtrain, num_round)
    predictions = model.predict(dvalid)
    score = r2_score(y_val, predictions)
    print("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}


In [10]:
def optimize(trials):
    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 1),
             'eta' : hp.quniform('eta', 0.0001, 0.5, 0.025),
             'max_depth' : sample(scope.int(hp.quniform('max_depth', 0, 10, 1))),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'eval_metric': 'rmse',
             'objective': 'reg:linear',
             'nthread' : 8,
             'silent' : 1
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=250)

    print(best)

In [ ]:
trials = Trials()

optimize(trials)

## Generate submission file

In [9]:
y_pred = model.predict(dtest)
output = pd.DataFrame({'id': x_test.index, 'y': y_pred})
output.to_csv('xgboost-depth{}-pca-ica.csv'.format(xgb_params['max_depth']), index=False)
